# Predictive model

In [ ]:
import sys
sys.path.append('D:/source/repos')
from utilities.std_imports import *

from datetime import datetime
from sklearn.neural_network import MLPRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score
import tensorflow.keras.models as tkm
import tensorflow.keras.layers as tkl
import trading.utils.SurfTrading as st
from mlfs.utils.LSTM import *
from ds_topics.utils.data_balance import *

### Model

In [ ]:
def get_LSTM_model(X_train):
    model = tkm.Sequential()
    model.add(tkl.LSTM(units=50, input_shape=(X_train.shape[1], X_train.shape[2])))
    model.add(tkl.Dense(1))
    model.compile(loss='mae', optimizer='adam')
    return model

def fit_model(model, X_train, y_train, X_test, y_test, verbose=0):
    history = model.fit(X_train, y_train, epochs=10, batch_size=72, validation_data=(X_test, y_test), verbose=verbose, shuffle=False)
    plt.plot(history.history['loss'], label='train')
    plt.plot(history.history['val_loss'], label='test')
    plt.legend()
    plt.show()
    
def predict(model, X_test, y_test, rows):
    pred = model.predict(X_test)
    res = pd.DataFrame(y_test[:,0], columns=['real'])
    res['pred'] = np.round(pred)
    return res.head(rows)

### Testing

#### Data Loading

In [ ]:
ts = st.get_rtdata('ACN')
plt.figure(figsize=(20, 10));
ts.plot();
pd.DataFrame(ts).head(20)

In [ ]:
ds = st.GenTrendDs(ts=ts, start=0, nHist=15, nFcst=15, tpVar=0.2, slVar=0.2, incFcst=False)
ds.to_csv('train.csv')
ds

In [ ]:
catlabel_dist(ds, 'Trend')

In [ ]:
numlabel_dist(ds, 'H1')